In [1]:
import os
import gensim
import numpy as np
import pandas as pd

from lxml import html
from matplotlib import pyplot as plt
from collections import Counter,defaultdict
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%matplotlib inline

In [2]:
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))


def normalize(text):  
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)


def tokenize(text):  
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


In [3]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1': texts_1, 'text_2': texts_2, 'label': classes})

In [4]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

In [5]:
data.head()

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_notnorm,text_2_notnorm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,полицейским разрешат стрелять на поражение по ...,полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,право полицейских на проникновение в жилище ре...,правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,президент египта ввел чрезвычайное положение в...,власти египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,вернувшихся из сирии россиян волнует вопрос тр...,самолеты мчс вывезут россиян из разрушенной сирии
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия,в москву из сирии вернулись 2 самолета мчс с р...,самолеты мчс вывезут россиян из разрушенной сирии


In [6]:
data.shape

(7227, 7)

## Нужные инструменты

In [26]:
def get_embedding(text, model, dim, mode, tfidf):
    text = text.split()

    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i, word in enumerate(words):
        try:
            v = model[word]
            if mode == 'sum':
                vectors[i] = v * (words[word] / total)
            else:
                vectors[i] = (v * (words[word] / total)) * tfidf[word]
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [24]:
def create_vectors(model, data1, data2, mode):
    dim = 50
    tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
    
    tfidf1 = tfidf.fit(data1).vocabulary_
    tfidf2 = tfidf.fit(data2).vocabulary_

    vec_1 = np.zeros((len(data1), dim))
    vec_2 = np.zeros((len(data2), dim))

    for i, text in enumerate(data1.values):
        vec_1[i] = get_embedding(text, model, dim, mode, tfidf1)

    for i, text in enumerate(data2.values):
        vec_2[i] = get_embedding(text, model, dim, mode, tfidf2)
        
    return vec_1, vec_2

In [9]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)

In [10]:
data_rt = pd.read_csv('news_texts.csv')
data_rt.dropna(inplace=True)

In [11]:
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)
fast_text = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)

## SVD

In [12]:
svd = TruncatedSVD(200)

In [13]:
svd_cv_1 = svd.fit_transform(cv.fit_transform(data['text_1_norm']))
svd_cv_2 = svd.fit_transform(cv.fit_transform(data['text_2_norm']))

In [14]:
svd_tfidf_1 = svd.fit_transform(tfidf.fit_transform(data['text_1_norm']))
svd_tfidf_2 = svd.fit_transform(tfidf.fit_transform(data['text_2_norm']))

## NMF

In [15]:
nmf = NMF(50)

In [16]:
nmf_cv_1 = nmf.fit_transform(cv.fit_transform(data['text_1_norm']))
nmf_cv_2 = nmf.fit_transform(cv.fit_transform(data['text_2_norm']))

In [17]:
nmf_tfidf_1 = nmf.fit_transform(tfidf.fit_transform(data['text_1_norm']))
nmf_tfidf_2 = nmf.fit_transform(tfidf.fit_transform(data['text_2_norm']))

## Word2Vec

In [27]:
w2v_1, w2v_2 = create_vectors(w2v, data['text_1_norm'], data['text_2_norm'], 'sum')

/home/deltamachine/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [28]:
w2v_tfidf_1, w2v_tfidf_2 = create_vectors(w2v, data['text_1_norm'], data['text_2_norm'], 'tfidf')

/home/deltamachine/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


## FastText

In [48]:
ft_1, ft_2 = create_vectors(fast_text, data['text_1_norm'], data['text_2_norm'], 'sum')
ft_notnorm_1, ft_notnorm_2 = create_vectors(fast_text, data['text_1_notnorm'], data['text_2_notnorm'], 'sum')

/home/deltamachine/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [49]:
ft_tfidf_1, ft_tfidf_2 = create_vectors(fast_text, data['text_1_norm'], data['text_2_norm'], 'tfidf')
ft_notnorm_tfidf_1, ft_notnorm_tfidf_2 = create_vectors(fast_text, data['text_1_notnorm'], data['text_2_notnorm'], 'tfidf')

/home/deltamachine/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


## Обучающая выборка

In [50]:
def calc_cosine(array1, array2):
    sims = []

    for vec1, vec2 in zip(array1, array2):
        cos_sim = cosine_distances(vec1.reshape(1, -1), vec2.reshape(1, -1))
        sims.append(cos_sim[0][0])
    
    return sims

In [62]:
def create_new_df(svd_cv_1, svd_cv_2, svd_tfidf_1, svd_tfidf_2, nmf_cv_1, nmf_cv_2,
                       nmf_tfidf_1, nmf_tfidf_2, w2v_1, w2v_2, w2v_tfidf_1, w2v_tfidf_2,
                       ft_1, ft_2, ft_tfidf_1, ft_tfidf_2, ft_notnorm_1, ft_notnorm_2,
                       ft_notnorm_tfidf_1, ft_notnorm_tfidf_2):
    
    svd_cv_sims = calc_cosine(svd_cv_1, svd_cv_2)
    svd_tfidf_sims = calc_cosine(svd_tfidf_1, svd_tfidf_2)
    nmf_cv_sims = calc_cosine(nmf_cv_1, nmf_cv_2)
    nmf_tfidf_sims = calc_cosine(nmf_tfidf_1, nmf_tfidf_2)
    w2v_sims = calc_cosine(w2v_1, w2v_2)
    w2v_tfidf_sims = calc_cosine(w2v_tfidf_1, w2v_tfidf_2)
    ft_sims = calc_cosine(ft_1, ft_2)
    ft_tfidf_sims = calc_cosine(ft_tfidf_1, ft_tfidf_2)
    ft_notnorm_sims = calc_cosine(ft_notnorm_1, ft_notnorm_2)
    ft_notnorm_tfidf_sims = calc_cosine(ft_notnorm_tfidf_1, ft_notnorm_tfidf_2)
    
    new_df = pd.DataFrame(data={'svd_cv': svd_cv_sims, 'svd_tfidf': svd_tfidf_sims,
                            'nmf_cv': nmf_cv_sims, 'nmf_tfidf': nmf_tfidf_sims,
                            'w2v': w2v_sims, 'w2v_tfidf': w2v_tfidf_sims,
                            'ft': ft_sims, 'ft_tfidf': ft_tfidf_sims,
                            'ft_notnorm': ft_notnorm_sims, 'ft_notnorm_tfidf': ft_notnorm_tfidf_sims})

    return new_df

In [63]:
new_df = create_new_df(svd_cv_1, svd_cv_2, svd_tfidf_1, svd_tfidf_2, nmf_cv_1, nmf_cv_2,
                       nmf_tfidf_1, nmf_tfidf_2, w2v_1, w2v_2, w2v_tfidf_1, w2v_tfidf_2,
                       ft_1, ft_2, ft_tfidf_1, ft_tfidf_2, ft_notnorm_1, ft_notnorm_2,
                       ft_notnorm_tfidf_1, ft_notnorm_tfidf_2)

new_df.head()

,ft,ft_notnorm,ft_notnorm_tfidf,ft_tfidf,nmf_cv,nmf_tfidf,svd_cv,svd_tfidf,w2v,w2v_tfidf
0,0.244800,0.189682,1.112563e+00,0.236303,0.962197,0.997646,1.023312,1.011008,0.077090,0.047405
1,0.183456,0.221199,1.776357e-15,0.425990,0.859799,0.999966,0.832068,0.982782,0.090184,0.335754
2,0.156042,0.277384,4.906398e-01,0.582511,0.968937,0.690889,1.140478,1.053396,0.047743,0.285524
3,0.377974,0.290506,1.281028e-02,0.153267,0.998754,0.975273,1.030974,1.017624,0.267040,0.153700
4,0.309268,0.430101,4.417429e-02,0.035608,0.988798,0.998602,0.935794,1.033440,0.077500,0.035003


## Обучение

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(random_state=55518)
mean_cvs = np.mean(cross_val_score(lr, new_df, data['label'], scoring='f1_micro', cv=5))

print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.544869581643414


## Подбор параметров

1) Был показатель 100 для TruncatedSVD, стал 50.

In [65]:
svd = TruncatedSVD(50)

svd_cv_1 = svd.fit_transform(cv.fit_transform(data['text_1_norm']))
svd_cv_2 = svd.fit_transform(cv.fit_transform(data['text_2_norm']))

svd_tfidf_1 = svd.fit_transform(tfidf.fit_transform(data['text_1_norm']))
svd_tfidf_2 = svd.fit_transform(tfidf.fit_transform(data['text_2_norm']))

new_df = create_new_df(svd_cv_1, svd_cv_2, svd_tfidf_1, svd_tfidf_2, nmf_cv_1, nmf_cv_2,
                       nmf_tfidf_1, nmf_tfidf_2, w2v_1, w2v_2, w2v_tfidf_1, w2v_tfidf_2,
                       ft_1, ft_2, ft_tfidf_1, ft_tfidf_2, ft_notnorm_1, ft_notnorm_2,
                       ft_notnorm_tfidf_1, ft_notnorm_tfidf_2)

mean_cvs = np.mean(cross_val_score(lr, new_df, data['label'], scoring='f1_micro', cv=5))
print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.5487476033286147


Качество улучшилось, но незначительно.
Изменим параметр NMF c 50 до 150 (оставив новые svd).

In [66]:
nmf = NMF(150)

nmf_cv_1 = nmf.fit_transform(cv.fit_transform(data['text_1_norm']))
nmf_cv_2 = nmf.fit_transform(cv.fit_transform(data['text_2_norm']))

nmf_tfidf_1 = nmf.fit_transform(tfidf.fit_transform(data['text_1_norm']))
nmf_tfidf_2 = nmf.fit_transform(tfidf.fit_transform(data['text_2_norm']))

new_df = create_new_df(svd_cv_1, svd_cv_2, svd_tfidf_1, svd_tfidf_2, nmf_cv_1, nmf_cv_2,
                       nmf_tfidf_1, nmf_tfidf_2, w2v_1, w2v_2, w2v_tfidf_1, w2v_tfidf_2,
                       ft_1, ft_2, ft_tfidf_1, ft_tfidf_2, ft_notnorm_1, ft_notnorm_2,
                       ft_notnorm_tfidf_1, ft_notnorm_tfidf_2)

mean_cvs = np.mean(cross_val_score(lr, new_df, data['label'], scoring='f1_micro', cv=5))
print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.544869583630642


Качество не улучшилось. Можно изменить параметр C у логрега.

In [70]:
lr_new = LogisticRegression(C=0.1)

mean_cvs = np.mean(cross_val_score(lr_new, new_df, data['label'], scoring='f1_micro', cv=5))
print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.5307523088458476


In [71]:
lr_new = LogisticRegression(C=0.01)

mean_cvs = np.mean(cross_val_score(lr_new, new_df, data['label'], scoring='f1_micro', cv=5))
print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.5217585371957033


Уменьшение параметра C снижает качество. Можно попробовать изменить penalty.

In [72]:
lr_new = LogisticRegression(penalty='l1')

mean_cvs = np.mean(cross_val_score(lr_new, new_df, data['label'], scoring='f1_micro', cv=5))
print('Среднее качество (f1 micro) на 5 фолдах = ', mean_cvs)

Среднее качество (f1 micro) на 5 фолдах =  0.5553875699543143


Качество улучшилось.